In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('../../Leagues/1st_2nd_tiers_top_5_leagues.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.sort_values(by=['year', 'country', 'tier', 'position'], ascending=[True, True, True, True], inplace=True)

In [33]:
df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
925,2004,ES,LaLiga,1,FC Barcelona,1.0,35,24.9,15,7800000,273030000,NaN,NaN,False
924,2004,ES,LaLiga,1,Real Madrid,2.0,34,25.9,10,8930000,303700000,NaN,NaN,False
931,2004,ES,LaLiga,1,Villarreal CF,3.0,38,24.9,10,1740000,66250000,NaN,NaN,False
929,2004,ES,LaLiga,1,Real Betis Balompié,4.0,32,26.6,8,2660000,85150000,NaN,NaN,False
935,2004,ES,LaLiga,1,RCD Espanyol Barcelona,5.0,29,26.1,8,1560000,45150000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,2024,L,2-Bundesliga,2,SV 07 Elversberg,NaN,26,26.4,4,655000,17030000,NaN,NaN,False
2558,2024,L,2-Bundesliga,2,Eintracht Braunschweig,NaN,28,26.0,10,520000,14550000,NaN,NaN,False
2559,2024,L,2-Bundesliga,2,SSV Ulm 1846,NaN,29,25.8,4,450000,13050000,NaN,NaN,True
2560,2024,L,2-Bundesliga,2,Preußen Münster,NaN,28,27.0,7,391000,10950000,NaN,NaN,False


In [34]:
df_spain = df[df['country'] == 'ES']
df_spain

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
925,2004,ES,LaLiga,1,FC Barcelona,1.0,35,24.9,15,7800000,273030000,NaN,NaN,False
924,2004,ES,LaLiga,1,Real Madrid,2.0,34,25.9,10,8930000,303700000,NaN,NaN,False
931,2004,ES,LaLiga,1,Villarreal CF,3.0,38,24.9,10,1740000,66250000,NaN,NaN,False
929,2004,ES,LaLiga,1,Real Betis Balompié,4.0,32,26.6,8,2660000,85150000,NaN,NaN,False
935,2004,ES,LaLiga,1,RCD Espanyol Barcelona,5.0,29,26.1,8,1560000,45150000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2024,ES,LaLiga2,2,Racing Ferrol,NaN,25,29.4,4,484000,12100000,NaN,NaN,False
1802,2024,ES,LaLiga2,2,CD Eldense,NaN,25,28.5,6,468000,11700000,NaN,NaN,False
1803,2024,ES,LaLiga2,2,FC Cartagena,NaN,21,28.9,5,514000,10800000,NaN,NaN,False
1804,2024,ES,LaLiga2,2,CD Mirandés,NaN,21,24.6,4,424000,8900000,NaN,NaN,False


In [35]:
df_spain_copy = df_spain.copy()

# Inicializar las columnas has_promoted y has_relegated con NaN
df_spain_copy['has_promoted'] = np.nan
df_spain_copy['has_relegated'] = np.nan

for idx, row in df_spain_copy.iterrows():
    year, team, current_tier = row['year'], row['team'], row['tier']
    
    if year == 2024:
        continue  # Para 2024 dejamos los valores como NaN
    
    next_year = df_spain_copy[(df_spain_copy['year'] == year + 1) & (df_spain_copy['team'] == team)]
    
    if not next_year.empty:  # Si existe registro del equipo en el año siguiente
        next_tier = next_year.iloc[0]['tier']
        
        if next_tier < current_tier:  # Promoción
            df_spain_copy.at[idx, 'has_promoted'] = True
            df_spain_copy.at[idx, 'has_relegated'] = False
        elif next_tier > current_tier:  # Descenso
            df_spain_copy.at[idx, 'has_promoted'] = False
            df_spain_copy.at[idx, 'has_relegated'] = True
        else:  # Mismo nivel
            df_spain_copy.at[idx, 'has_promoted'] = False
            df_spain_copy.at[idx, 'has_relegated'] = False
    else:  # El equipo no aparece en el año siguiente
        if current_tier == 2:  # Si estaba en tier 2 y desaparece, asumimos descenso
            df_spain_copy.at[idx, 'has_promoted'] = False
            df_spain_copy.at[idx, 'has_relegated'] = True
        else:  # Para tier 1 u otros casos, dejamos como NaN
            df_spain_copy.at[idx, 'has_promoted'] = np.nan
            df_spain_copy.at[idx, 'has_relegated'] = np.nan

df_spain_copy

C:\Users\Tore\AppData\Local\Temp\ipykernel_16896\1592466012.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_spain_copy.at[idx, 'has_promoted'] = False
C:\Users\Tore\AppData\Local\Temp\ipykernel_16896\1592466012.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_spain_copy.at[idx, 'has_relegated'] = False


,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
925,2004,ES,LaLiga,1,FC Barcelona,1.0,35,24.9,15,7800000,273030000,False,False,False
924,2004,ES,LaLiga,1,Real Madrid,2.0,34,25.9,10,8930000,303700000,False,False,False
931,2004,ES,LaLiga,1,Villarreal CF,3.0,38,24.9,10,1740000,66250000,False,False,False
929,2004,ES,LaLiga,1,Real Betis Balompié,4.0,32,26.6,8,2660000,85150000,False,False,False
935,2004,ES,LaLiga,1,RCD Espanyol Barcelona,5.0,29,26.1,8,1560000,45150000,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2024,ES,LaLiga2,2,Racing Ferrol,NaN,25,29.4,4,484000,12100000,NaN,NaN,False
1802,2024,ES,LaLiga2,2,CD Eldense,NaN,25,28.5,6,468000,11700000,NaN,NaN,False
1803,2024,ES,LaLiga2,2,FC Cartagena,NaN,21,28.9,5,514000,10800000,NaN,NaN,False
1804,2024,ES,LaLiga2,2,CD Mirandés,NaN,21,24.6,4,424000,8900000,NaN,NaN,False


In [36]:
test_df = df_spain_copy[df_spain_copy['year'] != 2024]
test_df

In [39]:
test_df.to_csv('./spain_df.csv')